# Laboratorio 6 Parte 1

### Reducción de dimensión: Selección de características

### 2019-I

#### Profesor: Julián D. Arias Londoño
#### julian.ariasl@udea.edu.co

## Estudiantes

#### Primer Integrante:
#### Segundo Integrante:

## Guía del laboratorio

En esta archivo va a encontrar tanto celdas de código cómo celdas de texto con las instrucciones para desarrollar el laboratorio.

Lea atentamente las instrucciones entregadas en las celdas de texto correspondientes y proceda con la solución de las preguntas planteadas.

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

## Indicaciones

Este ejercicio tiene como objetivo implementar varias técnicas de selección de características y usar SVM para resolver un problema de clasificación multiclase.

Para el problema de clasificación usaremos la siguiente base de datos: https://archive.ics.uci.edu/ml/datasets/Cardiotocography


Analice la base de datos, sus características, su variable de salida y el contexto del problema.

**Abstract:**
The dataset consists of measurements of fetal heart rate (FHR) and uterine contraction (UC) features on cardiotocograms classified by expert obstetricians.

**Data Set Information:**
2126 fetal cardiotocograms (CTGs) were automatically processed and the respective diagnostic features measured. The CTGs were also classified by three expert obstetricians and a consensus classification label assigned to each of them. Classification was both with respect to a morphologic pattern (A, B, C. ...) and to a fetal state (N, S, P). Therefore the dataset can be used either for 10-class or 3-class experiments.

**Attribute Information:**
LB - FHR baseline (beats per minute)

AC - # of accelerations per second

FM - # of fetal movements per second

UC - # of uterine contractions per second

DL - # of light decelerations per second

DS - # of severe decelerations per second

DP - # of prolongued decelerations per second

ASTV - percentage of time with abnormal short term variability

MSTV - mean value of short term variability

ALTV - percentage of time with abnormal long term variability

MLTV - mean value of long term variability

Width - width of FHR histogram

Min - minimum of FHR histogram

Max - Maximum of FHR histogram

Nmax - # of histogram peaks

Nzeros - # of histogram zeros

Mode - histogram mode

Mean - histogram mean

Median - histogram median

Variance - histogram variance

Tendency - histogram tendency

CLASS - FHR pattern class code (1 to 10)

NSP - fetal state class code (N=normal (1); S=suspect (2); P=pathologic (3))

Antes de iniciar a ejecutar las celdas, debe instalar la librería mlxtend que usaremos para los laboratorios de reducción de dimensión.
Para hacerlo solo tiene que usar el siguiente comando: sudo pip install mlxtend. También puede consultar la guía oficial de instalación
    de esta librería: https://rasbt.github.io/mlxtend/installation/

Analice y comprenda la siguiente celda de código donde se importan las librerías a usar y se carga la base de datos.

In [4]:
#!pip install mlxtend

In [2]:
from __future__ import division

import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import KFold
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import time

#cargamos la bd de entrenamiento
db = np.loadtxt('DB/DB_Fetal_Cardiotocograms.txt',delimiter='\t')  # Assuming tab-delimiter

X = db[:,0:22]

#Solo para dar formato a algunas variables
for i in range(1,7):
    X[:,i] = X[:,i]*1000

X = X
Y = db[:,22]

print('Dimensiones de la base de datos de entrenamiento. dim de X: ' + str(np.shape(X)) + '\tdim de Y: ' + str(np.shape(Y)))


Dimensiones de la base de datos de entrenamiento. dim de X: (2126, 22)	dim de Y: (2126,)


En la siguiente celda de código no tiene que completar nada. Analice, comprenda y ejecute el código y tenga en cuenta los resultados para completar la tabla que se le pide más abajo.

In [3]:
def classification_error(y_est, y_real):
    err = 0
    for y_e, y_r in zip(y_est, y_real):

        if y_e != y_r:
            err += 1

    return err/np.size(y_est)

#Para calcular el costo computacional
tiempo_i = time.time()

#Creamos el clasificador SVM. Tenga en cuenta que el problema es multiclase. 
clf = svm.SVC(decision_function_shape='ovr', kernel='rbf', C = 100, gamma=0.0001)

#Implemetamos la metodología de validación

Errores = np.ones(10)
j = 0
kf = KFold(n_splits=10)

for train_index, test_index in kf.split(X):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]  

    #Aquí se entran y se valida el modelo sin hacer selección de características
    
    ######
    
    # Entrenamiento el modelo.
    model = clf.fit(X_train,y_train)

    # Validación del modelo
    ypred = model.predict(X_test)
    
    #######

    Errores[j] = classification_error(ypred, y_test)
    j+=1

print("\nError de validación sin aplicar SFS: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))

print(('\n\nTiempo total de ejecución: ' + str(time.time()-tiempo_i)) + ' segundos.')


Error de validación sin aplicar SFS: 0.07712817787226504 +/- 0.05442325724156325


Tiempo total de ejecución: 0.5844535827636719 segundos.


## Ejercicio 1

1.1 Describa la metodología de validación que se está aplicando.

R/: Validación cruzada con 10 Folds


    
1.2 Con qué modelo se está resolviendo el problema planteado? Cuáles son los parámetros establecidos para el modelo?

R/: Maquina de soporte vectorial SVM y sus parametros son: decision_function_shape='ovr', kernel='rbf', C = 100, gamma=0.0001

## Ejercicio 2

En la siguiente celda, complete el código donde le sea indicado. Consulte la documentación oficial de la librería mlxtend para los métodos de selección de características. https://rasbt.github.io/mlxtend/user_guide/feature_selection/SequentialFeatureSelector/#sequential-feature-selector

In [5]:
#Feature Selection Function
#Recibe 4 parámetros: 1. el modelo (clf para nuestro caso), 2. el número de características final que se quiere alcanzar
#3. Si es forward (True), si es Backward False, 4. Si es es flotante (True), sino False
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

def select_features(modelo, n_features, fwd, fltg):
    
    sfs = SFS(modelo, 
           k_features=n_features, 
           forward=fwd,
           floating=fltg,
           verbose=1,
           scoring='accuracy',
           cv=0)    
    return sfs


#Para calcular el costo computacional
tiempo_i = time.time()

#Implemetamos la metodología de validación 

Errores = np.ones(10)
j = 0
kf = KFold(n_splits=10)
clf = svm.SVC(decision_function_shape='ovr', kernel='rbf', C = 100, gamma=0.0001)
for train_index, test_index in kf.split(X):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]  
   
    #Aquí se entrena y se valida el modelo haciendo selección de características con diferentes estrategias
    
    #Complete el código llamando el método select_features con los parámetros correspondientes para responder el
    #Ejercicio 3.1
    
    sf = select_features(clf, 7, False, False) #Complete el código aquí

    #Complete el código para entrenar el modelo con las características seleccionadas. Tenga en cuenta
    #la metodología de validación aplicada para que pase las muestras de entrenamiento correctamente.
    sf.fit(X_train, y_train)#Complete el código aquí
    
    Errores[j] = 1-sf.k_score_
    j+=1

print("\nError de validación aplicando SFS: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))

print("\nEficiencia en validación aplicando SFS: " + str(sf.k_score_*100) + "%" )

print ("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i) + " segundos.")

#print str(ypred)
#print str(y_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:    1.9s finished
Features: 21/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    1.9s finished
Features: 20/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    1.7s finished
Features: 19/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    1.7s finished
Features: 18/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    1.6s finished
Features: 17/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  17 out of  17 | elapsed:    1.2s finished
Features: 16/7[Parallel(

Features: 17/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  17 out of  17 | elapsed:    1.3s finished
Features: 16/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    1.3s finished
Features: 15/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    1.3s finished
Features: 14/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  14 out of  14 | elapsed:    1.7s finished
Features: 13/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:    1.5s finished
Features: 12/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    1.2s finished
Features: 

Features: 12/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.9s finished
Features: 11/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.7s finished
Features: 10/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.7s finished
Features: 9/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.7s finished
Features: 8/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.6s finished
Features: 7/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:    2.4s finished
Features: 21/


Error de validación aplicando SFS: 0.01991207385424808 +/- 0.0026168019256843054

Eficiencia en validación aplicando SFS: 98.27586206896551%


Tiempo total de ejecución: 203.69087266921997 segundos.


## Ejercicio 3

3.1 En la celda de código anterior, varíe los parámetros correspondientes al número de características a seleccionar (use 3, 7 y 10) y la estrategia a implementar (SFS, SBS, SFFS, SBFS), para que complete la siguiente tabla de resultados:


In [11]:
def implementation(n_features):
    #Para calcular el costo computacional
    tiempo_i = time.time()

    #Implemetamos la metodología de validación 

    Errores = np.ones(10)
    j = 0
    kf = KFold(n_splits=10)
    clf = svm.SVC(decision_function_shape='ovr', kernel='rbf', C = 100, gamma=0.0001)
    for train_index, test_index in kf.split(X):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]  

        #Aquí se entrena y se valida el modelo haciendo selección de características con diferentes estrategias

        #Complete el código llamando el método select_features con los parámetros correspondientes para responder el
        #Ejercicio 3.1

        sf = select_features(clf, n_features, False, False) #Complete el código aquí

        #Complete el código para entrenar el modelo con las características seleccionadas. Tenga en cuenta
        #la metodología de validación aplicada para que pase las muestras de entrenamiento correctamente.
        sf.fit(X_train, y_train)#Complete el código aquí

        Errores[j] = 1-sf.k_score_
        j+=1

    #print("\nError de validación aplicando SFS: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))

    #print("\nEficiencia en validación aplicando SFS: " + str(sf.k_score_*100) + "%" )

    #print ("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i) + " segundos.")
    tiempo = time.time()-tiempo_i
    return np.mean(Errores),np.std(Errores),tiempo
    #print str(ypred)
    #print str(y_test)

In [ ]:
array = [22,3,7,10,3,7,10,3,7,10,3,7,10]
tiempos = []
errores=[] 
stds =[]
for i in array:
    error,std, tiempo = implementation(i)
    tiempos.append(tiempo)
    errores.append(error) 
    stds.append(std)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:    1.9s finished
Features: 21/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    2.0s finished
Features: 20/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    1.6s finished
Features: 19/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    1.5s finished
Features: 18/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    1.5s finished
Features: 17/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  17 out of  17 | elapsed:    1.3s finished
Features: 16/3[Parallel(

Features: 10/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.6s finished
Features: 9/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.7s finished
Features: 8/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.6s finished
Features: 7/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.5s finished
Features: 6/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.4s finished
Features: 5/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.3s finished
Features: 4/3[P

Features: 17/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  17 out of  17 | elapsed:    2.2s finished
Features: 16/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    1.6s finished
Features: 15/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    1.2s finished
Features: 14/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  14 out of  14 | elapsed:    1.2s finished
Features: 13/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:    1.1s finished
Features: 12/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.9s finished
Features: 

Features: 5/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.4s finished
Features: 4/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.4s finished
Features: 3/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:    2.2s finished
Features: 21/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    1.9s finished
Features: 20/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    2.1s finished
Features: 19/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    1.5s finished
Features: 18/

Features: 12/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    1.0s finished
Features: 11/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    1.1s finished
Features: 10/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.7s finished
Features: 9/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.6s finished
Features: 8/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.6s finished
Features: 7/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:    2.0s finished
Features: 21/

...

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.8s finished
Features: 9/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.7s finished
Features: 8/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.7s finished
Features: 7/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:    2.0s finished
Features: 21/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    2.1s finished
Features: 20/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    1.7s finished
Features: 19/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    1.4s finished
Features: 19/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    1.3s finished
Features: 18/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    1.4s finished
Features: 17/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  17 out of  17 | elapsed:    1.1s finished
Features: 16/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    1.3s finished
Features: 15/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    1.1s finished
Features: 14/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(

Features: 21/10[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    1.6s finished
Features: 20/10[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    1.6s finished
Features: 19/10[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    1.4s finished
Features: 18/10[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    1.4s finished
Features: 17/10[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  17 out of  17 | elapsed:    1.1s finished
Features: 16/10[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    1.2s finished
Feat

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    1.8s finished
Features: 19/10[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    1.6s finished
Features: 18/10[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    1.4s finished
Features: 17/10[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  17 out of  17 | elapsed:    1.5s finished
Features: 16/10[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    1.3s finished
Features: 15/10[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    1.2s finished
Features: 14/10[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Par

Features: 6/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.5s finished
Features: 5/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.4s finished
Features: 4/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.4s finished
Features: 3/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:    1.7s finished
Features: 21/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    1.7s finished
Features: 20/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    1.6s finished
Features: 19/3

In [6]:
import pandas as pd
import qgrid
df_types = pd.DataFrame({
    'Tecnica' : pd.Series(['SVM sin selección','SVM + SFS','SVM + SFS','SVM + SFS','SVM + SBS','SVM + SBS','SVM + SBS','SVM + SFFS','SVM + SFFS','SVM + SFFS','SVM + SBFS','SVM + SBFS','SVM + SBFS']),
    '# de características seleccionadas' : pd.Series([22,3,7,10,3,7,10,3,7,10,3,7,10]),
   })
df_types["Error de validación"] = ""
df_types["IC(std)"] = ""
df_types["Tiempo de ejecución"] = ""

df_types.set_index(['Tecnica','# de características seleccionadas'], inplace=True)
df_types["Error de validación"] = errores
df_types["IC(std)"]= stds
df_types["Tiempo de ejecución"] = tiempos
#df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [7]:
qgrid_widget.get_changed_df()

Error de validación  \
Tecnica           # de características seleccionadas                       
SVM sin selección 22                                                       
SVM + SFS         3                                                        
                  7                                                        
                  10                                                       
SVM + SBS         3                                                        
                  7                                                        
                  10                                                       
SVM + SFFS        3                                                        
                  7                                                0.019   
                  10                                                       
SVM + SBFS        3                                                        
                  7                                                        
                  10                                                       

                                                     IC(std)  \
Tecnica           # de características seleccionadas           
SVM sin selección 22                                           
SVM + SFS         3                                            
                  7                                            
                  10                                           
SVM + SBS         3                                            
                  7                                            
                  10                                           
SVM + SFFS        3                                            
                  7                                    0.002   
                  10                                           
SVM + SBFS        3                                            
                  7                                            
                  10                                           

                                                     Tiempo de ejecución  
Tecnica           # de características seleccionadas                      
SVM sin selección 22                                                      
SVM + SFS         3                                                       
                  7                                                       
                  10                                                      
SVM + SBS         3                                                       
                  7                                                       
                  10                                                      
SVM + SFFS        3                                                       
                  7                                              107.9 s  
                  10                                                      
SVM + SBFS        3                                                       
                  7                                                       
                  10

3.2 Según la teoría vista en el curso, se está usando una función tipo filtro o tipo wrapper y cuál es?


R/: Es de tipo Wrapper ya que implemente un modelo de ML para la selección de características con base en el que mejor acurracy tenga.

3.3 Con los resultados de la tabla anterior haga un análisis de cuál es el mejor resultado teniendo en cuenta tanto la eficiencia en la clasificación como el costo computacional del modelo y la estrategia implementada.

R/:

3.4 Haga uso del atributo sf.k_feature\_idx\_ (deje evidencia del código usado para esto) para identificar cuáles fueron las características seleccionadas en el mejor de los resultados encontrados. No presente los indices de las características sino sus nombres y descripción.

R/:

3.5 De acuerdo a los resultados encontrados y la respuesta anterior, usted como ingeniero de datos que le puede sugerir a un médico que esté trabajando en un caso enmarcado dentro del contexto de la base de datos trabajada, para que apoye su diagnóstico?

R/: que se tome un tinto y no pierda el tiempo preguntando cosas demas 